In [5]:
import pandas as pd

# Charger le jeu de données
df = pd.read_csv('../../../Data/processed/data_pre_process.csv', index_col=0)

# Définition du chemin vers les images
path_to_images = '../../../Data/raw/images/image_train/'

# Création des noms de fichier des images
df['Nom image'] = ['image_' + str(imageid) + '_product_' + str(productid) + '.jpg' for imageid, productid in zip(df['imageid'], df['productid'])]

# Création des liens complets vers les images
df['lien'] = path_to_images + df['Nom image']

# Afficher les premières lignes pour comprendre la structure
display(df.head())

,designation,description,productid,imageid,prdtypecode,lang_designation,lang_description,designation_in_description_ratio,description_complete,lang_description_complete,description_complete_length,Nom image,lien
0,olivia personalisiertes notizbuch seiten punkt...,olivia personalisiertes notizbuch seiten punkt...,3804725264,1263597046,10,de,de,1.0,olivia personalisiertes notizbuch seiten punkt...,de,59,image_1263597046_product_3804725264.jpg,../../../Data/raw/images/image_train/image_126...
1,journal art marche salon asiatique paris jacqu...,journal art marche salon asiatique paris jacqu...,436067568,1008141237,2280,fr,fr,1.0,journal art marche salon asiatique paris jacqu...,fr,119,image_1008141237_product_436067568.jpg,../../../Data/raw/images/image_train/image_100...
2,grand stylet ergonomique bleu gamepad nintendo...,pilot style touch pen marque speedlink stylet ...,201115110,938777978,50,en,fr,1.0,grand stylet ergonomique bleu gamepad nintendo...,fr,567,image_938777978_product_201115110.jpg,../../../Data/raw/images/image_train/image_938...
3,peluche donald europe disneyland marionnette d...,peluche donald europe disneyland marionnette d...,50418756,457047496,1280,fr,fr,1.0,peluche donald europe disneyland marionnette d...,fr,50,image_457047496_product_50418756.jpg,../../../Data/raw/images/image_train/image_457...
4,guerre tuques,luc idées grandeur veut organiser jeu guerre b...,278535884,1077757786,2705,fr,fr,0.5,guerre tuques luc idées grandeur veut organise...,fr,118,image_1077757786_product_278535884.jpg,../../../Data/raw/images/image_train/image_107...


In [6]:
from skimage import io, color, exposure
from skimage.filters import threshold_otsu
from skimage.transform import resize
import numpy as np
import os

# Fonction pour améliorer le contraste d'une image
def enhance_contrast(image):
    image_enhanced = exposure.equalize_adapthist(image, clip_limit=0.03)
    return (image_enhanced * 255).astype(np.uint8)  # Convertir pour la sauvegarde en JPG

# Fonction pour supprimer les bords blancs d'une image avec buffer 10 pour test 
def remove_white_borders(image, buffer=10):
    gray_image = color.rgb2gray(image)
    thresh = threshold_otsu(gray_image)
    binary_image = gray_image < thresh
    # Trouver les coordonnées des pixels non-blancs (du sujet)
    coords = np.argwhere(binary_image)
    y0, x0 = coords.min(axis=0) - buffer  # Minus buffer pour une marge autour du sujet
    y1, x1 = coords.max(axis=0) + buffer  # Plus buffer pour une marge autour du sujet
    # Clipping des coordonnées pour éviter les valeurs hors limites
    y0, x0 = max(y0, 0), max(x0, 0)
    y1, x1 = min(y1, image.shape[0]), min(x1, image.shape[1])
    cropped_image = image[y0:y1, x0:x1]
    return cropped_image

def normalize_image(image):
    # Normaliser l'image pour que les valeurs des pixels soient entre 0 et 1
    return image / 255.0

# Chemins vers les dossiers source et destination
source_base_path = '../../../Data/raw/images/image_train'
destination_base_path = '../../../Data/processed/images/image_train'

# Taille cible pour les images redimensionnées, si nécessaire
target_size = (64, 64)  # Exemple de taille cible

# Dossiers de destination existent ?
if not os.path.exists(destination_base_path):
    os.makedirs(destination_base_path)

# Prétraiter et déplacer les images dans les dossiers de classes
for index, row in df.iterrows():
    # Construction du lien des images
    file = row['Nom image']  # Assurez-vous que cette colonne contient le nom de fichier correct
    prdtypecode = row['prdtypecode']  # La colonne avec les identifiants de classe
    
    source_path = os.path.join(source_base_path, file)
    destination_dir = os.path.join(destination_base_path, str(prdtypecode))
    destination_path = os.path.join(destination_dir, file)

    # Répertoire de classe existe ? Sinon, création 
    if not os.path.exists(destination_dir):
        os.makedirs(destination_dir, exist_ok=True)

    # L'image existe ? 
    if os.path.exists(source_path):
        # Chargement de l'image 
        image = io.imread(source_path)

        # Fonctions de pré-traitement
        image_contrasted = enhance_contrast(image)
        image_without_borders = remove_white_borders(image_contrasted)
        image_normalized = normalize_image(image_without_borders)
        image_resized = resize(image_normalized, target_size, anti_aliasing=True)

        # Sauvegarde de l'image prétraitée
        io.imsave(destination_path, (image_resized * 255).astype(np.uint8))
